In [1]:
from estnltk import Text
from estnltk.taggers import TaggerTester

# Tagger tester
`TaggerTester` class helps to create tests for taggers.

We create tests for the `NumberTagger`.

In [2]:
from estnltk.taggers import Tagger
from estnltk import Layer, Span
import regex as re


class NumberTagger(Tagger):
    """Tags numbers.

    """
    conf_param = ['regex']

    def __init__(self,
                 output_layer='numbers',
                 output_attributes=(),
                 input_layers=()           
                ):
        self.output_layer = output_layer
        self.output_attributes = output_attributes
        self.input_layers = input_layers
        self.regex = re.compile('-?\d+')

    def _make_layer(self, text, layers, status=None):
        layer = Layer(self.output_layer, text_object=text)
        for m in self.regex.finditer(text.text):
            layer.add_annotation((m.start(), m.end()))
        if isinstance(status, dict):
            status['NumberTagger message'] = self.output_layer + ' layer created successfully'
        return layer

Create an instance of the tagger.

In [3]:
tagger = NumberTagger()

Input file will contain a list of the Text objects in the estnltk json format.

In [4]:
input_file='input.json'

Target file will contain a list of the `Layer` objects in the estnltk json format.

In [5]:
target_file='target.json'

An instance of the `TaggerTester` will contain all the tests we create.

In [6]:
tester = TaggerTester(tagger=tagger, input_file=input_file, target_file=target_file)

To create a new test we need a `Text` object that can be tagged with the tagger. It means that the text object must have all of the input layers of the tagger. `NumberTagger` has no input layers so here we don't add any to the text object.

In [7]:
text = Text('')

To add test to the tester, use `add_test` method.

In [8]:
tester.add_test(annotation='empty text', text=text, expected_text=[])

`annotation` is a string that describes the test. `expected_text` is the text of the layer created by the tagger. If the `expected_text` doesn't match with the output of the tagger, then an `AssertionError` is raised and the test is not added to the tester.

Add one more test.

In [9]:
text = Text('-12,3')
tester.add_test(annotation='simple', text=text, expected_text=['-12', '3'])

Save the test data.

In [10]:
tester.save_input(overwrite=False)
tester.save_target(overwrite=False)

Created input texts file 'input.json'.
Created target layers file 'target.json'.


By default, the test data files are not overwritten.

In [11]:
tester.save_input()
tester.save_target()

Input texts file 'input.json' already exists. Use 'overwrite=True' to overwrite.
Target layers file 'target.json' already exists. Use 'overwrite=True' to overwrite.


The `run_tests` method runs all the tests up to the first failure. In case of failure, `run_tests` raises `AssertionError`.

In [12]:
tester.run_tests()

empty text PASSED
simple PASSED


Change the `NumberTagger` by modifying its regex and create a new tagger instance.

In [13]:
class NumberTagger(Tagger):
    """Tags numbers.

    """
    conf_param = ['regex']

    def __init__(self,
                 output_layer='numbers',
                 output_attributes=(),
                 input_layers=()           
                ):
        self.output_layer = output_layer
        self.output_attributes = output_attributes
        self.input_layers = input_layers
        self.regex = re.compile('-?\d')  # this regex is changed

    def _make_layer(self, text, layers, status=None):
        layer = Layer(self.output_layer, text_object=text)
        for m in self.regex.finditer(text.text):
            layer.add_annotation((m.start(), m.end()))
        if isinstance(status, dict):
            status['NumberTagger message'] = self.output_layer + ' layer created successfully'
        return layer


tagger = NumberTagger()

Load the tester from the files.

In [14]:
tester = TaggerTester(tagger=tagger, input_file=input_file, target_file=target_file).load()

Running the tests with the modified tagger raises the `AssertionError`.

In [15]:
try:
    tester.run_tests()
except AssertionError as e:
    print('error:', e)

empty text PASSED
simple FAILED
error: simple


`inspect_tests` is a generator method of broken tests and `diagnose` method explains the problem (yes, it needs improvements).

In [16]:
for test in tester.inspect_tests():
    print(test.annotation)
    print(test.diagnose())

simple
numbers layer spans differ


In [17]:
test

Test(simple, Text(text='-12,3'))

Example test function in the `pytest` file.

In [18]:
def test_tagger():
    input_file='input.json'
    target_file='target.json'
    tagger = NumberTagger()
    tester = TaggerTester(tagger, input_file, target_file).load()
    tester.run_tests()

For real examples see [test data creation](../../dev_documentation/testing) and 
[test files](../../estnltk/tests/test_taggers/test_standard_taggers).

Remove the test data files.

In [19]:
from os import remove

remove(input_file)
remove(target_file)